## Computing the percent of uninsured who have over the median household income

Pulled `HCOVANY` and `HHINCOME` from IPUMS for ACS 2019. That's `usa_00016.*` in the
data.

In [1]:
from ipumspy import readers

ddi_codebook = readers.read_ipums_ddi("../../data/ipums/usa_00016.xml")
ipums_df = readers.read_microdata(ddi_codebook, "../../data/ipums/usa_00016.dat.xz")

/Users/kwilso14/repo/brown/HX-AdvertisementValences-private/.venv/lib/python3.8/site-packages/ipumspy/readers.py:46: CitationWarning: Use of data from IPUMS is subject to conditions including that users should cite the data appropriately.
See the `ipums_conditions` attribute of this codebook for terms of use.
See the `ipums_citation` attribute of this codebook for the appropriate citation.
  warnings.warn(


In [2]:
ipums_df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,HHINCOME,PERNUM,PERWT,HCOVANY
0,2019,201901,1,2019010000088,11.0,2019000000011,220001,4,9999999,1,11.0,2
1,2019,201901,2,2019010000096,70.0,2019000000021,100001,3,9999999,1,70.0,1
2,2019,201901,3,2019010000153,20.0,2019000000031,110001,4,9999999,1,20.0,2
3,2019,201901,4,2019010000198,79.0,2019000000041,110001,3,9999999,1,79.0,2
4,2019,201901,5,2019010000205,53.0,2019000000051,270101,3,9999999,1,53.0,1


In [3]:
ddi_codebook.get_variable_info("HCOVANY").codes

{'No health insurance coverage': 1, 'With health insurance coverage': 2}

In [4]:
# Booleanize having insurance
ipums_df["has_insurance"] = ipums_df["HCOVANY"] != ddi_codebook.get_variable_info("HCOVANY").codes["No health insurance coverage"]


In [5]:
# How many people have insurance?
print(f'{((~ipums_df["has_insurance"]) * ipums_df["PERWT"]).sum():0,.0f}')

31,003,296


In [6]:
# The NA value is high, the bottom code is negative, and the top code doesn't exist, so
# take a `<` and then negate below in ultimate computation
#
# Median income for 2019 pulled from https://fred.stlouisfed.org/series/MEHOINUSA646N
ipums_df["below_median_income"] = (ipums_df["HHINCOME"] < 68703)
print(f'{ipums_df["below_median_income"].sum():,d}')

1,295,546


In [7]:
# Proportion of people who do not have insurance make _above_ median income
val = (
    (~ipums_df["has_insurance"] * ~ipums_df["below_median_income"] * ipums_df["PERWT"]).sum() / 
    (~ipums_df["has_insurance"] * ipums_df["PERWT"]).sum()
)
print(f"{val:0.03f}")

0.410
